# Analysis of Dopaminergic Cell Detector (DCD) model

## 0. Outline
This code deals with the automatic processing of raw data from mouse brains analysed with “Dopaminergic Cell Detector” model developed in Aiforia® Create. We typically start from Excel/CSV files collected in a local folder on the computer that is specified in the code. To automatically change the format of a series of files, refer to the Change_Name_Format_Input_Data.ipynb notebook. The code is developed to take into account that a mouse brain can be mounted over several slides. Slides for each animal are named identically, except for a numeric postfix denoting the slide number:'_S1', '_S2', etc. For this code it is required that mouse brain sections containing the substantia nigra from the same mouse are all mounted on the same '_S2' slide.

The present notebook is divided into 3 sections:

**1) Make the necessary functions for part 2 and 3**

**2) Automatic Analysis of N Slides (of which the name contains '_S2') of N Brains**

Here we automate the analysis of all N S2 slides of all N brains (1 slide per brain) in the folder with raw data. The approach is as follows:

1) We collect all the N names of the S2 raw data files in the folder and store them in a list.

2) We loop over these N slides belonging to the N brains and perform the following steps in each loop:

    a) We perform the data analysis steps.  
    b) We output the results to an excel file for this specific brain.
    
After each loop, we add the output of this specific brain to an overview table that will contain all results for all brains. After the last loop, this overview table is also exported to an excel file.

In this part of the code, we **do not** make more detailed **'transposed'** tables with information on each Substantia Nigra part. 

**3) Automatic Analysis of N Slides (of which the name contains '_S2') of N Brains after determining to which hemisphere they belong**

Here we add which Substantia Nigra regions are on each hemisphere, and compare the injected vs uninjected sides. Analysis occurs similar to section 2. In this part of the code, we also make more detailed **'transposed'** tables with information on each Substantia Nigra part for easier visualization.

## Part 1 - Make the necessary functions


### Part 1.1 - Load all necessary Python packages

In [ ]:
# Import the required Python packages
import pandas as pd                                # For data analysis with dataframes
import math                                        # To get the value for pi
import functools                                   # For higher-order functions that work on other functions
from IPython.display import display                # Enables the display of more than one dataframe per code cell
import numpy as np                                 # For data analysis
import glob                                        # To get all raw data file locations
import os                                          # To get all raw data file locations
pd.options.display.float_format = '{:.2f}'.format  # Display all numbers in dataframes with 2 decimals
import re                                          # Regular Expressions

### Part 1.2 - Data locations

**TO DO:** 
- Specify the format of the raw data and the raw data folder location, as well as some experimental parameters.
- Specify the file paths of the excel file containing your quality control revisions and the excel file mapping each brain region to a hemisphere.
- Specify the folder locations where you would like to collect the output excel files (for whole brain and hemisphere analysis).  

The format is: <font color='darkred'>r'file_location'</font> 

In [ ]:
# Specify what data format you want to use for your raw data: excel, csv or feather. Do this by uncommenting the data_format that you want.
# data_format = 'excel'
data_format = 'csv'
# data_format = 'feather'

# Specify the experimental parameters (section_thickness in micrometers) and locations:
# The spacing parameter refers to the serial section spacing interval. It's the interval at which you sample the brain volume for analysis, not the physical distance between each section. For example, if you have a spacing parameter of 10, you would take every 10th section for your analysis.  
spacing=5
section_thickness = 40
folder_raw_data = r'C:\Users\...\Raw_Data_DCD'
file_brainregions_to_replace =  r'C:\Users\...\Brainregions_To_Replace_DCD.xlsx'
file_brainregions_injected =  r'C:\Users\...\Brainregions_Hemisphere_DCD.xlsx'
folder_output_results = r'C:\Users\...\Results_Wholebrain_DCD'
folder_output_results_injected = r'C:\Users\...\Results_Hemisphere_DCD'


In [ ]:
# Make the output folders if they did not exist yet
if not os.path.isdir(folder_output_results):
    os.mkdir(folder_output_results)
if not os.path.isdir(folder_output_results_injected):
    os.mkdir(folder_output_results_injected)

### Part 1.3 - Function to load all image files that need to be analyzed

In [ ]:
def load_all_file_locations_S2(folder_raw_data):
    """
    Make a list of all file locations for S2 images present in the folder with all raw data files. There are only supposed to be S2 images in the folder, 
    so '_S2' does not have to be in the filename here.
    Output: list of all file locations for S2 images.    
    """

    if data_format == 'excel':
        all_raw_data_file_locations_S2 = glob.glob(os.path.join(folder_raw_data, "*.xlsx"))
    elif data_format == 'csv':
        all_raw_data_file_locations_S2 = glob.glob(os.path.join(folder_raw_data, "*.csv"))
    elif data_format == 'feather':
        all_raw_data_file_locations_S2 = glob.glob(os.path.join(folder_raw_data, "*.feather"))
    else:
        print('You did not specify a correct data-format in Part 1.2 and can expect some errors in the rest of the code')
        
    all_raw_data_file_locations_S2.sort()

    print('The location of all the raw data files = ')
    for file_location_S2 in all_raw_data_file_locations_S2:
        print(file_location_S2)

      
    return all_raw_data_file_locations_S2

### Part 1.4 - Function to load the file with corrections for the brainregions

In [ ]:
def load_data_brainregions_to_replace(file_brainregions_to_replace):
    """
    Load the file containing the corrections for brain regions that need to be replaced for each specific image.
    Output: cleaned dataframe with brain regions that need to be replaced for each image.
    """
    
    df_brainregions_to_replace_raw=pd.read_excel(file_brainregions_to_replace,
                                                 usecols=['Image', 'Brainregion_Wrong', 'Brainregion_Correct'],
                                                 dtype={'Image': 'str', 'Brainregion_Wrong': 'str', 'Brainregion_Correct': 'str'}
                                                )

    # Modify the dataframe to delete spaces that are by accident there, and put the brainregions in upper case 
    df_brainregions_to_replace=df_brainregions_to_replace_raw.copy()
    df_brainregions_to_replace['Image'] = df_brainregions_to_replace_raw['Image'].str.strip()
    df_brainregions_to_replace['Brainregion_Wrong'] = df_brainregions_to_replace_raw['Brainregion_Wrong'].str.upper().str.strip()
    df_brainregions_to_replace['Brainregion_Correct'] = df_brainregions_to_replace_raw['Brainregion_Correct'].str.upper().str.strip()

    #     print('The raw table of the brain regions to replace for each image = ')
    #     display(df_brainregions_to_replace_raw)

    print('The modified table of the brain regions to replace for each image = ')
    display(df_brainregions_to_replace)
    
    return df_brainregions_to_replace

### Part 1.5 - Function to load the file with which brainregions were injected


In [ ]:
def load_data_brainregions_injected(file_brainregions_injected):
    """
    Load the file specifying which brainregions were on the injected side for each specific image.
    Output: cleaned dataframe with brain regions that were injected for each image.
    """
    
    df_brainregions_injected_raw=pd.read_excel(file_brainregions_injected,
                                               usecols=['Image', 'Brainregion', 'Hemisphere'],
                                               dtype={'Image': 'str', 'Brainregion': 'str', 'Hemisphere': 'str'}
                                               )

    # Modify the dataframe to delete spaces that are by accident there, and put the brainregions in upper case 
    df_brainregions_injected=df_brainregions_injected_raw.copy()
    df_brainregions_injected['Image'] = df_brainregions_injected_raw['Image'].str.strip()
    df_brainregions_injected['Brainregion'] = df_brainregions_injected_raw['Brainregion'].str.upper().str.strip()
    df_brainregions_injected['Parent_Injected'] = df_brainregions_injected_raw['Hemisphere'].str.upper().str.strip()
    df_brainregions_injected['Daughter1_Injected'] = df_brainregions_injected_raw['Hemisphere'].str.upper().str.strip()
    df_brainregions_injected.drop(columns=['Hemisphere'], inplace=True)

    #     print('The raw table of the brain regions injected for each image = ')
    #     display(df_brainregions_injected_raw)

    print('The modified table of the brain regions injected for each image = ')
    display(df_brainregions_injected)
    
    return df_brainregions_injected

### Part 1.6 - Function to load dataframe and clean it


In [ ]:
def dataframe_cleaning(file_location, df_brainregions_to_replace):
    """
    Load the specific file location in a dataframe and clean it with df_brainregions_to_replace.
    Output: loaded and cleaned dataframe with some additional calculated values.
    """
    # Load file with raw data for image S2 of this specific brain
    if data_format == 'excel':
        df_1=pd.read_excel(file_location,
                           usecols=['Image', 'Parent area name', 'Area/object name', 'Class label', 'Area (μm²)', 
                                    'Class confidence (%)','Circumference (µm)'],
                           dtype={'Image': 'str', 'Parent area name': 'str', 'Area/object name': 'str', 'Class label': 'str', 
                                   'Area (μm²)': 'float64', 'Class confidence (%)' : 'float64', 'Circumference (µm)': 'float64' },
                           keep_default_na = True)
    elif data_format == 'csv':
        df_1=pd.read_csv(file_location, sep='\t',
                         usecols=['Image', 'Parent area name', 'Area/object name', 'Class label', 'Area (μm²)', 
                                    'Class confidence (%)','Circumference (µm)'],
                         dtype={'Image': 'str', 'Parent area name': 'str', 'Area/object name': 'str', 'Class label': 'str', 
                                 'Area (μm²)': 'float64', 'Class confidence (%)' : 'float64', 'Circumference (µm)': 'float64' },
                         keep_default_na = True)
    elif data_format == 'feather':
        df_1=pd.read_feather(file_location) 
        dtype_dictionary = {'Image': 'object', 'Parent area name': 'object', 'Area/object name': 'object', 'Class label': 'object', 
                            'Area (μm²)': 'float64', 'Class confidence (%)' : 'float64', 'Circumference (µm)': 'float64' }
        df_1=df_1.astype(dtype_dictionary)
    else:
        print('You did not specify a correct data-format in Part 1.2 and can expect some errors in the rest of the code')
        
        
    # Get the image name out of the file_path (getting image name from dataframe first column is hard because some are empty, 
    # and getting from filename makes more sense anyway) and change some field based on the recipe. 
    full_name = os.path.basename(file_location)
    file_name = os.path.splitext(full_name)
    image_name = file_name[0]
    print('The present image=', image_name)

    # Make sure the image name across the whole first column is correct
    df_1['Image']=image_name
    
    # Delete the rows with an empty 'Parent area name' or empty Area (μm²) or Area/object name or Class label
    df_1.dropna(subset =['Parent area name', 'Area (μm²)', 'Area/object name', 'Class label'] , how='any', inplace=True)
 
    # Put all columns in capitals to never make mistakes against capitalization
    df_1['Parent area name'] = df_1['Parent area name'].str.upper()
    df_1['Area/object name'] = df_1['Area/object name'].str.upper()
    df_1['Class label']      = df_1['Class label'].str.upper()
       
    print('The full raw data =')
    display(df_1)

    # Determine the dictionary of brain regions that should be replaced for this specific image
    df_brainregions_to_replace = df_brainregions_to_replace[df_brainregions_to_replace['Image']==image_name]
    dict_brainregions_to_replace= pd.Series(df_brainregions_to_replace.Brainregion_Correct.values, index=df_brainregions_to_replace.Brainregion_Wrong).to_dict()

    print('The dictionary of brain regions to replace for this specific image', image_name, 'is', dict_brainregions_to_replace)

    # Replace the value in the rows that have a Parent area name or Area/object name that is in list_brainregions_replace
    df_2 = df_1.copy()
    df_2['Parent area name'] = df_1['Parent area name'].replace(dict_brainregions_to_replace, regex=False)
    df_2['Area/object name'] = df_1['Area/object name'].replace(dict_brainregions_to_replace, regex=False)

    # Create a column 'Parent area name merged' and 'Area/object name merged' where the numbers are deleted from these columns:
    df_2['Parent area name merged'] = df_2['Parent area name'].str.replace('\\d+', '', regex=True).str.strip()
    df_2['Area/object name merged'] = df_2['Area/object name'].str.replace('\\d+', '', regex=True).str.strip()

    # We delete the rows that 
    # - have an area > 450 and class label = TH Positive
    # - have an area  in [400, 450] or [81, 95] and confidence < 60 and class label = TH Positive
    # - have an area < 81 and class label = TH Positive
    # Side note: Pandas between function is inclusive
    df_3 = df_2[ ~( (df_2['Area (μm²)'] > 450) & (df_2['Class label'] == 'TH POSITIVE') )
                 &
                 ~( (df_2['Area (μm²)'].between(400, 450)) & (df_2['Class confidence (%)'] < 60 ) & (df_2['Class label'] == 'TH POSITIVE') )
                 &
                 ~( (df_2['Area (μm²)'].between(81, 95)) & (df_2['Class confidence (%)'] < 60 ) & (df_2['Class label'] == 'TH POSITIVE') )
                 &
                 ~( (df_2['Area (μm²)'] < 81) & (df_2['Class label'] == 'TH POSITIVE') )
               ]    
    
    # Calculate the Area/Perimeter (μm) and the circularity
    df_4 = df_3.copy()
    df_4['Area/Perimeter (μm)'] = df_4['Area (μm²)']/df_4['Circumference (µm)']
    df_4['Circularity'] = (4 * math.pi * df_4['Area (μm²)'])/ (df_4['Circumference (µm)'])**2
    
    # Show the full updated dataframe:
    print('The new table with "Area/Perimeter (μm)" and "Circularity" ')
    display(df_4)
    
    return df_4


### Part 1.7 - Function to make hierarchical dataframes


In [ ]:
# This function will not be used for TH-Cell detector code, as the hierarchy does not extend till daughter 3. 
# We just have 1 type of parent (Tissue parent detector for Nigra 1, 2, 3 etc) with 1 type of daughter 1 (Substantia nigra 1, 2, 3 etc) 
# and 1 type of daughter 2 (TH Positive 963, TH Positive 2111 etc)
def make_hierarchy(df):
    """ 
    Here we make the hierarchical structure of the data in the dataframe more clear. 
    The field 'Parent area name' is always the parent of the 'Area/object name' in the same row. 
    The area in the row always belongs to the 'Area/object name'.
    Output: four dataframes in which gradually more hierarchy is added.
    """

    # The rows with the top parent (= BRAIN TISSUE X) are the rows that don't have an own Parent area name
    df_parent_almost = df[df['Parent area name'].isna()]
    dict_parent={'Area/object name':'Parent name', 'Area/object name merged': 'Parent name merged', 'Area (μm²)': 'Area Parent (μm²)',
                'Area/Perimeter (μm)': 'Area/Perimeter Parent (μm)', 'Circularity': 'Circularity Parent'}
    df_parent=df_parent_almost.rename(columns=dict_parent)
    df_parent.drop(columns=['Parent area name', 'Class label', 'Parent area name merged','Area/Perimeter Parent (μm)', 'Circularity Parent' ], inplace=True)

    # Then we add the first daughter = the daughter of the top parents
    df_parent_daughter1_almost=df_parent.merge(df[['Parent area name', 'Area/object name', 'Area/object name merged', 'Area (μm²)']], left_on='Parent name', right_on='Parent area name', how='inner')
    dict_daughter1 = {'Parent area name': 'Parent name copy', 'Area/object name':'Daughter1', 
                      'Area/object name merged': 'Daughter1 merged', 'Area (μm²)': 'Area Daughter1 (μm²)'}

    df_parent_daughter1_almost2=df_parent_daughter1_almost.rename(columns=dict_daughter1)
    # Groupby is needed because there now can be for instance 2 Striatum 4's 
    # (one of them originated from e.g. changing Amygdala 1 to Striatum 4 in the brainregion corrections)
    # We need to turn this Striatum 4 into a unique row because otherwise we will double in the next join when making df_parent_daughter2
    df_parent_daughter1=df_parent_daughter1_almost2.groupby(['Daughter1'], as_index=False).agg(
        {'Image': 'first', 'Parent name': 'first', 'Area Parent (μm²)': 'first',
         'Parent name merged': 'first', 'Parent name copy': 'first', 'Daughter1': 'first',
         'Daughter1 merged': 'first', 'Area Daughter1 (μm²)': 'sum'})

    # Then we add the second daughter = the daughter of daughter 1
    df_parent_daughter2_almost=df_parent_daughter1.merge(df[['Parent area name', 'Area/object name', 'Area/object name merged', 'Area (μm²)', 'Area/Perimeter (μm)' , 'Circularity' ]], left_on='Daughter1', right_on='Parent area name', how='inner')
    dict_daughter2 = {'Parent area name': 'Daughter1 copy','Area/object name':'Daughter2', 
                      'Area/object name merged': 'Daughter2 merged', 'Area (μm²)': 'Area Daughter2 (μm²)',
                      'Area/Perimeter (μm)': 'Area/Perimeter Daughter2 (μm)', 'Circularity': 'Circularity Daughter2'}
    df_parent_daughter2=df_parent_daughter2_almost.rename(columns=dict_daughter2)


#     print('Original df')
#     display(df)
#     print('df_parent')
#     display(df_parent)
#     print('df_parent_daughter1')
#     display(df_parent_daughter1)
#     print('df_parent_daughter2')
#     display(df_parent_daughter2)
    
    return df_parent, df_parent_daughter1, df_parent_daughter2

### Part 1.8 - Function to calculate all information for all parents without considering the hemisphere

In [ ]:
def all_calculations(df1, df2, groupby_column1='Parent area name merged', groupby_column2='Area/object name merged'):
    """
    Make the main calculations (areas, counts...) based on 2 dataframes (df1 and df2, but df1=df2 for the calculations without 'injected'), 
    and based on a groupby columns that can be chosen.
    Output: dataframe with all calculations.
    """

    # Count the number of rows for each parent area name merged 
    df_counts_merged = df1.value_counts(groupby_column1, sort=True).rename_axis('Merged area name').reset_index(name='Counts')
    # print('The number of rows for each Parent area name merged =')
    # display(df_counts_merged)
    
    # Count the total area of each Area/object name merged (e.g. Amygdala 1 + Amygdala 7 + ... area)
    df_total_region_area_merged = df2.groupby(groupby_column2).sum()['Area (μm²)'].rename_axis('Merged area name').reset_index(name='Total Region Area (μm²)')
    # print('The total region area of each Area/object name merged')
    # display(df_total_region_area_merged)

    # Calculate the total Area (μm²) of the cells belonging to each Parent area name merged
    df_total_cell_area_merged = df1.groupby(groupby_column1).sum(numeric_only=True)['Area (μm²)'].rename_axis('Merged area name').reset_index(name='Total Cell Area (μm²)')
    # print('The total Area (μm²) of the cells belonging to each Parent area name merged =')
    # display(df_total_cell_area_merged)
    
    # Calculate the average Area (μm²) of the cells belonging to each Parent area name merged
    df_average_cell_area_merged = df1.groupby(groupby_column1).mean(numeric_only=True)['Area (μm²)'].rename_axis('Merged area name').reset_index(name='Average Cell Area (μm²)')
    # print('The average Area (μm²) of the cells belonging to each Parent area name merged =')
    # display(df_average_cell_area_merged)
   
    # Calculate the average Area/Perimeter (μm) of the cells belonging to each Parent area name merged
    df_average_area_perimeter_merged = df1.groupby(groupby_column1).mean(numeric_only=True)['Area/Perimeter (μm)'].rename_axis('Merged area name').reset_index(name='Average Area/Perimeter (μm)')
    # print('The average Area/Perimeter (μm) of the cells belonging to each Parent area name merged =')
    # display(df_average_area_perimeter_merged)

    # Calculate the average circularity of the cells belonging to each Parent area name merged
    df_average_circularity_merged = df1.groupby(groupby_column1).mean(numeric_only=True)['Circularity'].rename_axis('Merged area name').reset_index(name='Average Circularity')
    # print('The average Circularity of the cells belonging to each Parent area name merged =')
    # display(df_average_circularity_merged)

    dfs_to_merge = [df_counts_merged, df_total_region_area_merged, df_total_cell_area_merged, df_average_cell_area_merged, 
                    df_average_area_perimeter_merged, df_average_circularity_merged]
    df_all_calcs_merged  = functools.reduce(lambda left, right: pd.merge(left,right,on='Merged area name', how='outer'), dfs_to_merge)

    # Put all calculated results together
    df_all_calcs_merged['Extrapolated Cell Count']       = df_all_calcs_merged['Counts']*spacing
    df_all_calcs_merged['Cells/Region Area (per μm²)']   = df_all_calcs_merged['Counts']/df_all_calcs_merged['Total Region Area (μm²)']
    df_all_calcs_merged['Cells/Region Volume (per μm³)'] = df_all_calcs_merged['Cells/Region Area (per μm²)']/section_thickness
    df_all_calcs_merged['Cells/Region Area (mm²)']       = df_all_calcs_merged['Cells/Region Area (per μm²)']*1000000
    df_all_calcs_merged['Cells/Region Volume (mm³)']     = df_all_calcs_merged['Cells/Region Volume (per μm³)']*1000000000
    
    df_all_calcs_merged.drop(columns=['Cells/Region Area (per μm²)', 'Cells/Region Volume (per μm³)'], inplace=True)
    df_all_calcs_merged.sort_values('Merged area name',inplace=True)
    
    print('The total Calculations of each Daugher1 merged:')
    display(df_all_calcs_merged)
    
    return df_all_calcs_merged

### Part 1.9 - Function to calculate all information for all parents without the specification injected/uninjected (but for the hemisphere part of the code)


In [ ]:
def all_calculations_parent2(df1, df_brainregions_injected, groupby_column1='Parent area name'):
    """
    Make the main calculations (areas, counts...) based on 1 dataframe and df_brainregions_injected, 
    and based on a groupby column that can be chosen.
    Output: dataframe with all calculations without the specification injected/uninjected.
    """
    
    # Count the number of rows for each parent area name (Substantia nigra 1, Substantia nigra 2 ... except for the ones containing TISSUE)
    df_counts = df1[~df1[groupby_column1].str.contains('TISSUE')].value_counts(groupby_column1, sort=True).reset_index(name='Counts')

    # Calculate the average area of the cells belonging to each Parent area name (except for the ones containing TISSUE)
    df_average_area = df1[~df1[groupby_column1].str.contains('TISSUE')].groupby(groupby_column1).mean(numeric_only=True)['Area (μm²)'].rename_axis('Parent area name').reset_index(name='Average Cell Area (μm²)')

    # Calculate the average Area/Perimeter of the cells belonging to each Parent area name  (except for the ones containing TISSUE)
    df_average_perimeter = df1[~df1[groupby_column1].str.contains('TISSUE')].groupby(groupby_column1).mean(numeric_only=True)['Area/Perimeter (μm)'].rename_axis('Parent area name').reset_index(name='Average Area/Perimeter (μm)')

    # Calculate the average circularity of the cells belonging to each Parent area name  (except for the ones containing TISSUE)
    df_average_circularity = df1[~df1[groupby_column1].str.contains('TISSUE')].groupby(groupby_column1).mean(numeric_only=True)['Circularity'].rename_axis('Parent area name').reset_index(name='Average Circularity')
    # display(df_average_circularity)
    
    # Calculate the Region Area of all SUBSTANTIA NIGRA, which is already given as Area in the row where Area/object name = Substantia nigra X 
    df_region_area=df1[['Image', 'Area/object name', 'Area (μm²)']].copy()
    df_region_area.rename(columns={'Area/object name': 'Parent area name', 'Area (μm²)': 'Total Region Area (μm²)'}, inplace=True)
    df_region_area2=df_region_area[df_region_area['Parent area name'].str.contains('SUBSTANTIA NIGRA')]
    # display(df_region_area2)
    
    # Put all calculated results together
    dfs_to_merge = [df_counts, df_average_area, df_average_perimeter, df_average_circularity, df_region_area2]
    df_all_calcs  = functools.reduce(lambda left, right: pd.merge(left,right,on=groupby_column1, how='outer'), dfs_to_merge)
    # display(df_all_calcs)
    
    # We add the information about (un)injected to df_all_calcs
    df_all_calcs_total= df_brainregions_injected.merge(df_all_calcs, left_on=['Image', 'Brainregion'], right_on=['Image', 'Parent area name'], how='inner')
    df_all_calcs_total.drop(columns=['Brainregion', 'Daughter1_Injected'], inplace=True)
    print('Full dataframes of calculations per region with injected/uninjected specification added')
    display(df_all_calcs_total) 
  
    return df_all_calcs_total

### Part 1.10 - Function to calculate all information for all parents with the specification injected/uninjected


In [ ]:
def all_calculations_injected(df1, df_brainregions_injected, groupby_column1='Parent_Injected', groupby_column2='Daughter1_Injected'):
    """
    Make the main calculations (areas, counts...) based on 1 dataframe and df_brainregions_injected, 
    and based on a groupby column that can be chosen.
    Output: dataframe with all calculations with the specification injected/uninjected.
    """

    # We add the information about (un)injected to all the raw data. One time we merge on parent area name, the other time on Area/object name
    df_injected_parent= df_brainregions_injected.merge(df1, left_on=['Image', 'Brainregion'], right_on=['Image', 'Parent area name'], how='inner')
    df_injected_parent.drop(columns=['Brainregion', 'Daughter1_Injected', 'Area/object name'], inplace=True)

    df_injected_object= df_brainregions_injected.merge(df1, left_on=['Image', 'Brainregion'], right_on=['Image', 'Area/object name'], how='inner')
    df_injected_object.drop(columns=['Brainregion', 'Parent_Injected'], inplace=True)

    # Count the number of rows for each Parent_Injected
    df_counts_injected = df_injected_parent.value_counts(groupby_column1, sort=True).reset_index(name='Counts')

    # Calculate the average area of the cells belonging to each Parent_Injected
    df_average_area_injected = df_injected_parent.groupby(groupby_column1).mean(numeric_only=True)['Area (μm²)'].rename_axis(groupby_column1).reset_index(name='Average Cell Area (μm²)')

    # Calculate the average Area/Perimeter of the cells belonging to each Parent_Injected
    df_average_perimeter_injected = df_injected_parent.groupby(groupby_column1).mean(numeric_only=True)['Area/Perimeter (μm)'].rename_axis(groupby_column1).reset_index(name='Average Area/Perimeter (μm)')

    # Calculate the average circularity of the cells belonging to each Parent_Injected
    df_average_circularity_injected = df_injected_parent.groupby(groupby_column1).mean(numeric_only=True)['Circularity'].rename_axis(groupby_column1).reset_index(name='Average Circularity')

    # Calculate the Total Region Area for the injected and uninjected side, and replace column name Daughter1_Injected by Parent_Injected for easier merge later
    df_region_area_injected =df_injected_object.groupby(groupby_column2).sum()['Area (μm²)'].rename_axis(groupby_column1).reset_index(name='Total Region Area (μm²)')


    # Put all calculated results together
    dfs_to_merge = [df_counts_injected, df_average_area_injected, df_average_perimeter_injected, df_average_circularity_injected, df_region_area_injected]
    df_all_calcs_injected  = functools.reduce(lambda left, right: pd.merge(left,right,on=groupby_column1, how='outer'), dfs_to_merge)

    df_all_calcs_injected['Extrapolated Cell Count']          = df_all_calcs_injected['Counts']*spacing
    df_all_calcs_injected['Cells/Region Area (per μm²)'] = df_all_calcs_injected['Counts']/df_all_calcs_injected['Total Region Area (μm²)']
    df_all_calcs_injected['Cells/Region Volume (per μm³)']      = df_all_calcs_injected['Cells/Region Area (per μm²)']/section_thickness
    df_all_calcs_injected['Cells/Region Area (mm²)'] = df_all_calcs_injected['Cells/Region Area (per μm²)']*1000000
    df_all_calcs_injected['Cells/Region Volume (mm³)']      = df_all_calcs_injected['Cells/Region Volume (per μm³)']*1000000000

    df_all_calcs_injected.sort_values(by=[groupby_column1], ascending=False, inplace=True)
    df_all_calcs_injected=df_all_calcs_injected[[groupby_column1, 'Counts', 'Extrapolated Cell Count', 'Average Cell Area (μm²)', 
                                                 'Average Area/Perimeter (μm)', 'Average Circularity', 'Total Region Area (μm²)', 
                                                 'Cells/Region Area (mm²)', 'Cells/Region Volume (mm³)']]
    print('Calculations on injected/uninjected regions')
    display(df_all_calcs_injected)

    return df_all_calcs_injected

### Part 1.11 - Function to calculate all transposed information of each Substantia Nigra X region


In [ ]:
def all_calculations_transposed(df1, df2, filter_column1='Parent_Injected'):
    """
    Calculate the Counts, Total Region Area and Average Cell Area of each Substantia Nigra X region, in a transposed table. Add the Extrapolated Cell Count.
    """

    df_short = df1[['Parent_Injected', 'Parent area name', 'Counts', 'Total Region Area (μm²)', 'Average Cell Area (μm²)']]
    df_short2= df2[['Parent_Injected', 'Counts', 'Total Region Area (μm²)', 'Average Cell Area (μm²)']]

    # For the uninjected cells
    dictionary_SN={}
    for region in ['SUBSTANTIA NIGRA UNINJECTED', 'SUBSTANTIA NIGRA INJECTED']:
        inj_uninj=region.split()[-1]   # This is either UNINJECTED or INJECTED
        df_short_transp = df_short[df_short[filter_column1] == region].transpose()

        df_short2_transp = df_short2[df_short2[filter_column1] == region].transpose()
    
        df_counts=pd.concat([df_short_transp, df_short2_transp], axis=1)
        df_counts.columns = df_counts.iloc[1]    # Change the column names nicely to SUBSTANTIA NIGRA 1 and so
        df_counts.rename(columns={np.nan: inj_uninj + " Total"}, inplace=True) 
        df_counts.rename(columns=lambda x: re.sub('SUBSTANTIA NIGRA ','SN', x), inplace=True) # Change the column names nicely from SUBSTANTIA NIGRA 1 to SN1
        df_counts.drop([filter_column1, 'Parent area name'], inplace=True)  # Delete superfluous rows
        df_counts[inj_uninj + ' Extrapolated Cell Count'] = df_counts[inj_uninj + " Total"]*spacing
        print(f'Transposed {region} regions with total counts')
        display(df_counts)
        dictionary_SN[region]=df_counts

    return dictionary_SN

## Part 2 - Automatic Wholebrain Analysis of all N S2 Slides of all N Brains


In [ ]:
%%time   
# For curiosity we measure the time the code in this cell takes to run

# Load the modified file with brain regions to replace/delete for each specific image 
df_brainregions_to_replace=load_data_brainregions_to_replace(file_brainregions_to_replace)

# Extract the file names that contain '_S2' in the file name. These are the N first images of the N unique brains.
all_raw_data_file_locations_S2= load_all_file_locations_S2(folder_raw_data)

# We initiate a counter to keep track in which loop we are below:
count = 0

# Loop over all the S2 pictures in the raw_data folder
for file_location_S2 in all_raw_data_file_locations_S2:
    count = count +1 # Counts the loop; first loop: counter = 1
    
    # Get the image name out of the file_path (getting image name from dataframe first column is hard because some are empty)
    full_name = os.path.basename(file_location_S2)
    file_name = os.path.splitext(full_name)
    image_name_S2 = file_name[0]
    
     
    # Do the S2 data cleaning, making use of the functions defined above
    print('\n Analysis of ', file_location_S2)
    df_S2_final = dataframe_cleaning(file_location_S2, df_brainregions_to_replace)

    # Do all the calculations, making use of the functions defined above. 
    df_S2_all_calcs_merged = all_calculations(df_S2_final, df_S2_final)
    print('All calculations together for S2 for ', file_location_S2)
    display(df_S2_all_calcs_merged)
     
    # Output the results to an excel file that is created in the output folder specified at the beginning of this notebook.
    output_file_name = image_name_S2 + '_Results.xlsx'
    output_file_location = os.path.join(folder_output_results, output_file_name)
 
    with pd.ExcelWriter(output_file_location) as writer:
        df_S2_all_calcs_merged.to_excel(writer, sheet_name='Results', index=False, float_format = "%.3f")

    
    # For the overview excel file, only the df_S2_all_calcs_merged dataframe is needed. 
    # We will make 1 overview excelfiles with a few tabpages that we store in dictionary_overview_dataframes:
    # dictionary_overview_dataframes = {Total Region area: df, Extrapolated Cell Count:df, Cells/Region Area:df, .... }
    
    # In the first loop we initiate an empty overview dictionary that will be filled with dataframes. 
    if count==1:
        dictionary_overview_dataframes={}

    # Prepare the dataframes that are needed for the overview excel file: choose the needed columns,
    # and rename the header of the column with the values to the image_name 
    list_calculation_results=['Counts', 'Extrapolated Cell Count', 
                              'Average Cell Area (μm²)', 'Total Region Area (μm²)', 
                              'Cells/Region Area (mm²)', 'Cells/Region Volume (mm³)',
                              'Total Cell Area (μm²)', 
                              'Average Area/Perimeter (μm)', 'Average Circularity', 
                             ]
    
    # print('list_calculation_results = ', list_calculation_results)
    brainregions_not_needed = ['TISSUE PARENT DETECTOR FOR NIGRA', 'TH POSITIVE']
    for calculation_result in list_calculation_results:
        df_all_calcs_merged_calculation= df_S2_all_calcs_merged[['Merged area name', calculation_result]].copy()
        df_all_calcs_merged_calculation = df_all_calcs_merged_calculation[~df_all_calcs_merged_calculation['Merged area name'].isin(brainregions_not_needed)]
        df_all_calcs_merged_calculation.rename(columns={calculation_result: image_name_S2}, inplace=True)
    
        if count==1:
            # In the first loop we fill the empty overview dictionary with a dataframe with the values calculated in loop 1  
            dictionary_overview_dataframes[calculation_result]  = df_all_calcs_merged_calculation.copy()

        elif count > 1 :
            # In the subsequent loops we will add the values of those loops to the dataframes in the overview dictionary
            dictionary_overview_dataframes[calculation_result] = dictionary_overview_dataframes[calculation_result].merge(df_all_calcs_merged_calculation, how='outer', on='Merged area name')

    # At the end, we delete some of the dataframes, so they cannot be used in the next loop
    try:
        del(df_S2_final)
        del(df_S2_all_calcs_merged)
    except:
        pass
    

# After the for loops, we print the final overview tables
# Output the final overview tables to an excel file Overview_THCells_Results.xlsx that is created in the output folder specified at the beginning of this notebook
output_file_name_overview = os.path.join(folder_output_results, 'Overview_THCells_Results.xlsx')
    
list_calculation_results=['Counts', 'Extrapolated Cell Count', 
                          'Average Cell Area (μm²)', 'Total Region Area (μm²)', 
                          'Cells/Region Area (mm²)', 'Cells/Region Volume (mm³)',
                          'Total Cell Area (μm²)', 
                          'Average Area/Perimeter (μm)', 'Average Circularity', 
                         ]
    
with pd.ExcelWriter(output_file_name_overview) as writer: 
    for calculation_result in list_calculation_results:
        calculation_result_clean = calculation_result.replace('/', ' per ').replace('Volume', 'Vol')
        
        print(f'Overview dataframe with all {calculation_result_clean} for all brains')
        display(dictionary_overview_dataframes[calculation_result])

        dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=False, float_format = "%.3f")


## Part 3 - Automatic Hemisphere Analysis of all N S2 Slides of all N Brains (injected vs uninjected)


In [ ]:
%%time   
# For curiosity we measure the time the code in this cell takes to run

# Load the modified file with brain regions to replace/delete for each specific image 
df_brainregions_to_replace=load_data_brainregions_to_replace(file_brainregions_to_replace)

# Load the modified file with hemisphere analysis for each specific image 
df_brainregions_injected=load_data_brainregions_injected(file_brainregions_injected)

# Extract the file names that contain '_S1' in the file name. These are the N first images of the N unique brains.
all_raw_data_file_locations_S2= load_all_file_locations_S2(folder_raw_data)

# We initiate a counter to keep track in which loop we are below:
count = 0

# Loop over all the S2 pictures in the raw_data folder
for file_location_S2 in all_raw_data_file_locations_S2:
    count = count +1 # Counts the loop; first loop: counter = 1

    # Get the image name out of the file_path 
    full_name = os.path.basename(file_location_S2)
    file_name = os.path.splitext(full_name)
    image_name_S2 = file_name[0]
    
    # Do the S2 data cleaning, making use of the functions defined above
    print('\n Analysis of ', file_location_S2)
    df_S2_final = dataframe_cleaning(file_location_S2, df_brainregions_to_replace)
    # These columns are obsolete in this part of the code:
    df_S2_final.drop(columns=['Parent area name merged', 'Area/object name merged' ], inplace=True)

    # Do all the calculations, making use of the functions defined above. 
    df_S2_all_calcs = all_calculations_parent2(df_S2_final, df_brainregions_injected)
    df_S2_all_calcs_injected = all_calculations_injected(df_S2_final, df_brainregions_injected)
    dictionary_S2_SN = all_calculations_transposed(df_S2_all_calcs, df_S2_all_calcs_injected)
    df_counts_uninjected = dictionary_S2_SN['SUBSTANTIA NIGRA UNINJECTED']
    df_counts_injected   = dictionary_S2_SN['SUBSTANTIA NIGRA INJECTED']

    # Output the results to an excel file that is created in the output folder specified at the beginning of this notebook
    output_file_name = image_name_S2 + '_Results.xlsx'
    output_file_location = os.path.join(folder_output_results_injected, output_file_name)
 
    with pd.ExcelWriter(output_file_location) as writer:
        df_S2_all_calcs.to_excel(writer, sheet_name='All Areas Results', index=False, float_format = "%.3f")
        df_counts_uninjected.to_excel(writer, sheet_name='Counts Horizontal', startrow=0,  index=True, float_format = "%.3f")
        df_counts_injected.to_excel(writer, sheet_name='Counts Horizontal',  startrow=5, index=True, float_format = "%.3f")
        df_S2_all_calcs_injected.to_excel(writer, sheet_name='Injected Results', index=False, float_format = "%.3f")
    
    ############################
    #   Prepare overview file  #
    ############################
    
    # For the overview files, we need to concatenate the counts horizontally. For this we change the SN column names 
    # to 'U1', 'U2, 'U3'.. and 'I1', 'I2', 'I3' because for some animals, SN7 can be on the injected side, 
    # for other animals it can be on the uninjected side.

    # Reset the column headers, such that the columns Sn14, SN7.. become the first row (with index 'Parent area name'), 
    # and the column headers just become 0, 1, 2, 3.
    # Also, remove the words 'UNINJECTED Extrapolated Cell Count' and 'UNINJECTED Extrapolated Cell Count' in the first row then
    df_counts_uninjected2= df_counts_uninjected.T.reset_index().T
    df_counts_injected2  = df_counts_injected.T.reset_index().T
    df_counts_uninjected2.replace({'UNINJECTED Total': np.nan, 'UNINJECTED Extrapolated Cell Count': np.nan}, inplace=True)
    df_counts_injected2.replace({'INJECTED Total': np.nan, 'INJECTED Extrapolated Cell Count': np.nan}, inplace=True)

    # Now give the column headers the desired name: 'U1', 'U2, 'U3'.. and 'I1', 'I2', 'I3'
    column_names_uninjected = ['U'+ str(x) for x in range(1, len(df_counts_uninjected2.columns)-1)] + ['U_Total', 'U_Extrapolated'] 
    column_names_injected   = ['I'+ str(x) for x in range(1, len(df_counts_injected2.columns)-1)] + ['I_Total', 'I_Extrapolated']
    
    df_counts_uninjected2.columns = column_names_uninjected
    df_counts_injected2.columns   = column_names_injected

    # Now concatenate the uninjected and injected data horizontally, and rename the indices so that the animal name is contained in them
    df_counts_horizontal_almost = pd.concat([df_counts_uninjected2,df_counts_injected2], axis=1)
    df_counts_horizontal = df_counts_horizontal_almost.filter(items=['Parent area name', 'Counts', 'Total Region Area (μm²)'], axis=0)
    df_counts_horizontal.rename(index={'Parent area name': 'Parent area name ' + image_name_S2,  
                                       'Counts': 'Counts ' + image_name_S2,  
                                       'Total Region Area (μm²)' : 'Total Region Area (μm²) ' + image_name_S2}, 
                                inplace=True)
    # Lastly, calculate the loss between the uninjected and injected counts and areas:
    a = pd.to_numeric(df_counts_horizontal['I_Total'], errors='coerce')
    b = pd.to_numeric(df_counts_horizontal['U_Total'], errors='coerce')
    df_counts_horizontal['Loss'] = 100*(1-a/b)
   
    print('Transposed injected and uninjected regions horizontally merged')
    display(df_counts_horizontal)


    # For the overview excel file, only the df_S2_all_calcs_injected dataframe is needed. 
    # We will make 1 overview excelfiles with a few tabpages that we store in dictionary_overview_dataframes:
    # dictionary_overview_dataframes = {Cell Count Overview : df, Average Cell Area (μm²): df, Total Region area: df, Extrapolated Cell Count:df, Cells/Area:df, .... }
    
    # In the first loop we initiate an empty overview dictionary that will be filled with dataframes. 
    if count==1:
        dictionary_overview_dataframes={}
        dictionary_overview_dataframes['Cell Count Overview']= df_counts_horizontal
    elif count > 1 : 
        dictionary_overview_dataframes['Cell Count Overview']  = pd.concat([dictionary_overview_dataframes['Cell Count Overview'], df_counts_horizontal])   

     
    # Prepare the dataframes that are needed for the overview excel file: choose the needed columns,
    # and rename the header of the column with the values to the image_name 
    list_calculation_results=['Extrapolated Cell Count', 'Average Cell Area (μm²)', 'Total Region Area (μm²)', 
                               'Cells/Region Area (mm²)', 'Cells/Region Volume (mm³)', 'Average Area/Perimeter (μm)', 'Average Circularity']
    
    # print('list_calculation_results = ', list_calculation_results)
     
    for calculation_result in list_calculation_results:
        df_S2_all_calcs_injected_calculation= df_S2_all_calcs_injected[['Parent_Injected', calculation_result]].copy()
        df_S2_all_calcs_injected_calculation.rename(columns={calculation_result: image_name_S2}, inplace=True)
        
        if count==1:
            # In the first loop we fill the empty overview dictionary with a dataframe with the values calculated in loop 1  
            dictionary_overview_dataframes[calculation_result]  = df_S2_all_calcs_injected_calculation.copy()
            
        elif count > 1 :
            # In the subsequent loops we will add the values of those loops to the dataframes in the overview dictionary
            dictionary_overview_dataframes[calculation_result] = dictionary_overview_dataframes[calculation_result].merge(df_S2_all_calcs_injected_calculation, how='outer', on='Parent_Injected')

    # At the end, we delete some of the dataframes, to ensure they cannot be used in the next loop
    del(df_S2_final)
    del(df_S2_all_calcs_injected)

                
# After the for loops, we print the final overview tables

# Start by rearranging the columns of the dataframe dictionary_overview_dataframes['Cell Count Overview']: 
# first  all Uninjected columns, then all Injected columns
columns_to_order = dictionary_overview_dataframes['Cell Count Overview'].columns
columns_0 = ['Loss']
columns_1 = [x for x in columns_to_order if 'U' in x and 'U_' not in x]   # Columns U1, U2, U3 ...
columns_1.sort(key= lambda x: float(x[1:]))  # to sort based on the number in it, and make sure that the order is U1, U2 ... U9, U10... instead of U1, U10, U2...
columns_2 = ['U_Total', 'U_Extrapolated']
columns_3 = [x for x in columns_to_order if 'I' in x and 'I_' not in x]   # Columns I1, I2, I3 ...
columns_3.sort(key= lambda x: float(x[1:]))
columns_4 = ['I_Total', 'I_Extrapolated']
columns_ordered = columns_0 + columns_1 + columns_2 + columns_3 + columns_4
print(columns_ordered)

# Now reorder the columns of the dataframe:
dictionary_overview_dataframes['Cell Count Overview'] = dictionary_overview_dataframes['Cell Count Overview'][columns_ordered]

# Also make a new dataframe that contains only the loss column, and only for the rows that contain the Counts of each animal
df=dictionary_overview_dataframes['Cell Count Overview']
df_loss_intensities=df[df.index.str.contains('Counts')]
dictionary_overview_dataframes['Cell Count Loss'] = df_loss_intensities['Loss']

# Output the final overview tables to an excel file Overview_THCells_Hemisphere_Results.xlsx that is created in the output folder specified at the beginning of this notebook
output_file_name_overview = os.path.join(folder_output_results_injected, 'Overview_THCells_Hemisphere_Results.xlsx')

list_calculation_results=['Cell Count Loss', 'Cell Count Overview', 'Extrapolated Cell Count', 'Average Cell Area (μm²)', 'Total Region Area (μm²)', 
                          'Cells/Region Area (mm²)', 'Cells/Region Volume (mm³)', 'Average Area/Perimeter (μm)', 'Average Circularity']
    
with pd.ExcelWriter(output_file_name_overview) as writer: 
    for calculation_result in list_calculation_results:
        calculation_result_clean = calculation_result.replace('/', ' per ').replace('Volume', 'Vol')
        
        print(f'Overview dataframe with all {calculation_result_clean} for all brains')
        display(dictionary_overview_dataframes[calculation_result])
        if calculation_result == 'Average Circularity': 
            dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=False, float_format = "%.4f")
        elif calculation_result in ['Cell Count Overview', 'Cell Count Loss']: 
            dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=True, float_format = "%.3f")
        else:
            dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=False, float_format = "%.3f")
